In [24]:
import pandas as pd

* Prendo il dataset risultante dalla fase di analisi e preprocessing.

In [25]:
ds = pd.read_csv("DataSet/games1.csv")
print(ds.columns.tolist())

['required_age', 'achievements', 'platform_count', 'is_free', 'price_log', 'trimester', 'success_class_encoded', 'top1_genre_freq', 'top2_genre_freq', 'genre_Indie', 'genre_Action', 'genre_Adventure', 'genre_Casual', 'genre_Free To Play', 'genre_RPG', 'genre_Strategy', 'genre_Simulation', 'genre_Early Access', 'genre_Massively Multiplayer', 'genre_Sports', 'genre_Racing', 'cat_Captions available', 'cat_Co-op', 'cat_Cross-Platform Multiplayer', 'cat_Family Sharing', 'cat_Full controller support', 'cat_In-App Purchases', 'cat_Includes Source SDK', 'cat_Includes level editor', 'cat_LAN Co-op', 'cat_LAN PvP', 'cat_MMO', 'cat_Multi-player', 'cat_Online Co-op', 'cat_Online PvP', 'cat_PvP', 'cat_Shared/Split Screen', 'cat_Shared/Split Screen Co-op', 'cat_Shared/Split Screen PvP', 'cat_Single-player', 'cat_Steam Trading Cards', 'publishers_freq', 'developer_freq', 'pub_top_Big Fish Games', 'pub_top_Conglomerate 5', 'pub_top_Daedalic Entertainment', 'pub_top_Devolver Digital', 'pub_top_DigiPen 

In [26]:
ds.to_csv('DataSet/games2.csv', index=False)

# Inizio fase di Addestramento
* divido la variabile target dal dataset

In [27]:
y = ds['success_class_encoded']
X = ds.drop(columns=['success_class_encoded'])

print(ds.columns.tolist())
print(y.head())

['required_age', 'achievements', 'platform_count', 'is_free', 'price_log', 'trimester', 'success_class_encoded', 'top1_genre_freq', 'top2_genre_freq', 'genre_Indie', 'genre_Action', 'genre_Adventure', 'genre_Casual', 'genre_Free To Play', 'genre_RPG', 'genre_Strategy', 'genre_Simulation', 'genre_Early Access', 'genre_Massively Multiplayer', 'genre_Sports', 'genre_Racing', 'cat_Captions available', 'cat_Co-op', 'cat_Cross-Platform Multiplayer', 'cat_Family Sharing', 'cat_Full controller support', 'cat_In-App Purchases', 'cat_Includes Source SDK', 'cat_Includes level editor', 'cat_LAN Co-op', 'cat_LAN PvP', 'cat_MMO', 'cat_Multi-player', 'cat_Online Co-op', 'cat_Online PvP', 'cat_PvP', 'cat_Shared/Split Screen', 'cat_Shared/Split Screen Co-op', 'cat_Shared/Split Screen PvP', 'cat_Single-player', 'cat_Steam Trading Cards', 'publishers_freq', 'developer_freq', 'pub_top_Big Fish Games', 'pub_top_Conglomerate 5', 'pub_top_Daedalic Entertainment', 'pub_top_Devolver Digital', 'pub_top_DigiPen 

In [28]:
ds.to_csv('DataSet/games2.csv', index=False)
y.to_csv('DataSet/target.csv', index=False)

* Divido in set di addestramento e di test (train\test) per valutare il modello.

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
# stratify mantiene le proporzioni delle classi nel train/test
# 20% dei dati per il test

## Definizione modello Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=500,       # numero alberi
    max_depth=None,
    min_samples_split=5,   # minimo campioni per split
    class_weight="balanced", # gestisce sbilanciamento
    random_state=42,
    n_jobs=-1           # usa tutti i core CPU
)

## Addestramento

In [31]:
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', min_samples_split=5,
                       n_estimators=500, n_jobs=-1, random_state=42)

### Salvo modello

In [32]:
import joblib
joblib.dump(model, "Modello/rf_model_future_game.pkl")

['Modello/rf_model_future_game.pkl']

## Valutazione performance

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      2147
           1       0.78      0.74      0.76      2655
           2       0.49      0.36      0.42      1088
           3       0.48      0.27      0.34       169

    accuracy                           0.75      6059
   macro avg       0.64      0.59      0.60      6059
weighted avg       0.73      0.75      0.73      6059

[[2113   16   11    7]
 [ 363 1960  322   10]
 [ 160  499  397   32]
 [  18   30   76   45]]


* analisi importanza delle variabili che pesano di più nella predizione

In [34]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
print(feat_importances.sort_values(ascending=False).head(20))


price_log                      0.142917
achievements                   0.113289
is_free                        0.073704
publishers_freq                0.067316
cat_Family Sharing             0.053345
cat_Steam Trading Cards        0.046697
developer_freq                 0.045031
trimester                      0.043643
top2_genre_freq                0.033615
platform_count                 0.028000
genre_Free To Play             0.023859
top1_genre_freq                0.022631
cat_Full controller support    0.017468
cat_Multi-player               0.016617
genre_Adventure                0.016245
genre_Action                   0.015809
required_age                   0.015743
genre_Simulation               0.015491
genre_Casual                   0.015406
genre_RPG                      0.015200
dtype: float64


In [35]:
#print(ds['achievements'].value_counts())
print(ds['price_log'].value_counts())

price_log
0.000000    13915
2.396986     1671
1.790091     1652
0.688135     1495
3.044046     1484
            ...  
1.479329        1
1.738710        1
1.266948        1
5.303255        1
0.756122        1
Name: count, Length: 292, dtype: int64
